In [103]:
from splinter import Browser #sign in function
import splinter
from selenium.common.exceptions import WebDriverException

browser = Browser('chrome')
url = "https://zh-tw.facebook.com/login/"
browser.visit(url)
username =input('username:')
password =input('password:')
search_input = browser.find_by_xpath('//*[@id="email"]')[0]
search_input.fill(username)
search_input = browser.find_by_xpath('//*[@id="pass"]')[0]
search_input.fill(password)
browser.find_by_xpath('//*[@id="loginbutton"]').first.click()

try:
    search_input = browser.find_by_xpath('//*[@id="approvals_code"]')[0]
    phonecheck = input('phonecheck_code:')
    search_input.fill(phonecheck)
    browser.find_by_xpath('//*[@id="checkpointSubmitButton"]').first.click()
    browser.find_by_xpath('//*[@id="u_0_0"]/div/div[2]/ul/li[2]/div/label/strong').first.click()
    browser.find_by_xpath('//*[@id="checkpointSubmitButton"]').first.click()
except splinter.exceptions.ElementDoesNotExist:
    pass

phonecheck_code:398459


In [70]:
def get_friends_list(url_input):#連線到某個人讀取他的朋友清單
    import time 
    from bs4 import BeautifulSoup
    import re
    import requests
    success = 0 #checking connection is succeeded or not
    count = 0 #define how many times we have tried to connect
    times_of_trying = 3 #the limited number we want to try
    friends_list = []
    while (success == 0 and count<times_of_trying) : #if not success to connect try few times
        try:
            browser.visit(url_input)
            if browser.find_by_text('Edit Profile') != []:
                browser.find_by_text('Friends')[0].click()
            else:
                browser.find_by_text('Friends')[1].click()
                
            soup_for_while = BeautifulSoup(browser.html, 'lxml') 
            select_condition = soup_for_while.select('#timeline-medley > div > div.mbm._5vf.sectionHeader._4khu > div.uiHeader > div > div > h3')
            
            count_for_while = 0
            while select_condition == [] and count_for_while<200:
                browser.execute_script('window.scrollTo(0, document.body.scrollHeight)')
                soup_for_while = BeautifulSoup(browser.html, 'lxml') 
                select_condition = soup_for_while.select('#timeline-medley > div > div.mbm._5vf.sectionHeader._4khu > div.uiHeader > div > div > h3')
                time.sleep(0.2)
                count_for_while += 1

                
            success = 1 #if succeeded define success connection
            
            soup = BeautifulSoup(browser.html, 'lxml') 
            section_friends = soup.select('#collection_wrapper_2356318349')[0].select('div')[0].select('ul')
            
#             if count>5 :
            for friends in section_friends: #find the friends list and save it to friend_list
                for friend in friends.select('li'):
                    if len(friend.select('div > div > div > div > div > div > a')) != 0:
                        if friend.select('div > div > div > div > div > div > a')[0].text == 'FriendFriends':
                            friends_list.append(friend.select('div > div > div > div > div > div > a')[1]['href'].replace('?fref=pb&hc_location=friends_tab',''))
#                             print(friend.select('div > div > div > div > div > div > a')[1].text)
#                             print(friend.select('div > div > div > div > div > div > a')[1]['href'].replace('?fref=pb&hc_location=friends_tab',''))

                        else:
                            friends_list.append(friend.select('div > div > div > div > div > div > a')[0]['href'].replace('?fref=pb&hc_location=friends_tab',''))
#                             print(friend.select('div > div > div > div > div > div > a')[0].text)
#                             print(friend.select('div > div > div > div > div > div > a')[0]['href'].replace('?fref=pb&hc_location=friends_tab',''))
        except splinter.exceptions.ElementDoesNotExist:
            count += 1
            time.sleep(0.5)

        except requests.ConnectionError:
            success = 1  

        except (AttributeError,IndexError): #if user does not permit us to see his firends try few times
            count += 1
            time.sleep(0.5)
            if count==times_of_trying:
                pass
        
    return friends_list

In [71]:
import os
data = "data"
if not os.path.exists(data):
    os.makedirs(data)

friendList = []

def read_frends_csv():
    global friendList
    with open("./data/friends.csv","r") as file:
        friendList = file.read().split()
        
def to_frends_csv():
    with open("./data/friends.csv","w") as file:
        for f in friendList:
            file.write(f+"\n")

try:        
    read_frends_csv()
    start_number = len(friendList) -1
except (FileNotFoundError,IndexError):
    friendList.append("https://www.facebook.com/v78203")
    start_number = len(friendList) -1

try:
    while True:
        each = friendList[start_number]
        for element in get_friends_list(each):
            if element not in friendList:
                friendList.append(element)
            if len(friendList)%10000 == 0:
                with open("./data/friends_copy.csv","w") as file:
                    for f in friendList:
                        file.write(f+"\n")
        start_number += 1

except (WebDriverException,KeyboardInterrupt) as e:
    to_frends_csv()

In [3]:
def get_likes_list(url_input):#連線到某個人抓他按讚的粉絲頁
    import time 
    from bs4 import BeautifulSoup
    import re
    import requests

    success = 0  #checking connection is succeeded or not
    count = 0  #define how many times we have tried to connect
    likes_list = []
    
    while (success == 0 and count<4) :
        try:
            browser.visit(url_input) #go user's page and find his likes
            browser.find_by_text('More').first.click()
            browser.find_by_text('Likes').first.click()
            
            soup_for_while = BeautifulSoup(browser.html, 'lxml') 
            select_condition = soup_for_while.select('#timeline-medley > div > div.mbm._5vf.sectionHeader._4khu > div.uiHeader > div > div > h3')
            
            count_for_while = 0
            while select_condition == [] and count_for_while<100:
                browser.execute_script('window.scrollTo(0, document.body.scrollHeight)')
                soup_for_while = BeautifulSoup(browser.html, 'lxml') 
                select_condition = soup_for_while.select('#timeline-medley > div > div.mbm._5vf.sectionHeader._4khu > div.uiHeader > div > div > h3')
                time.sleep(0.2)
                count_for_while += 1
                
            success = 1
            
            soup = BeautifulSoup(browser.html, 'lxml') #find the likes list and save it to likes_list
            
            
            for link in soup.select('#collection_wrapper_2409997254 > div')[0].select('ul > li > div > div > div > div'):
                each_like = {}
                each_like['like_name'] = link.select('div > div > a')[0].text
                each_like['like_url'] = link.select('div > div > a')[0]['href']
                each_like['like_category'] = link.select('div > div')[1].text
                likes_list.append(each_like)
            
        except splinter.exceptions.ElementDoesNotExist:
#             print(url_input,'\n','user do not permit likes reading','\n','or wrong url')
            success = 1
        except Exception as ee:
#             print(ee)
            count += 1
            
    return likes_list

In [ ]:
import json
import os
import sys

data = "data" #make the data direction if not exist
if not os.path.exists(data):
    os.makedirs(data)

likesList = []
likesList_copy = []

def read_friends_csv(countLikesCrawling): #read the friends from the file 
    i = 0
    with open("./data/friend_part_2.csv","r") as file:
        friendList = []
        for line in file:
            if i in range(countLikesCrawling,countLikesCrawling+100):
                friendList.append(line)
            elif i>countLikesCrawling+100:
                break
            i += 1
    return friendList
        
def read_progress_csv():
    with open("./data/progress.csv","r") as file:
        count = int(file.read())
    return count
        
def to_likes_csv(likesList):
    with open("./data/likes.csv","a") as file:
        for f in likesList:
            file.write(f+"\n")

def to_copy_likes_csv(likesList):
    with open("./data/likes_copy.csv","a") as file:
        for f in likesList:
            file.write(f+"\n")   

def save_progress(countLikesCrawling):
    with open("./data/progress.csv","w") as file:
        file.write(str(countLikesCrawling))


        
try:
    countLikesCrawling = read_progress_csv() #read progress, if from the first time set it as 0
except (FileNotFoundError,IndexError):
    countLikesCrawling = 0
                
try:        
    friendList = read_friends_csv(countLikesCrawling) #from progress read 100 friendslist
except (FileNotFoundError,IndexError):
    print('please check the friends list file')
    
try:    
    while True:
        for link in friendList:
            data = {'user_url':link , 'user_likes':get_likes_list(link)}
            jdata = json.dumps(data,ensure_ascii=False)
            likesList.append(jdata)
            likesList_copy.append(jdata)
            countLikesCrawling += 1
            printstring = ('now is on number '+ str(countLikesCrawling) + ' crawling')
            sys.stdout.write('\r' + printstring)
            if countLikesCrawling%50 == 0:
                to_copy_likes_csv(likesList_copy)
                save_progress(countLikesCrawling)
                likesList_copy = []
        friendList = read_friends_csv(countLikesCrawling)
            
except (WebDriverException,KeyboardInterrupt) as e:
    to_likes_csv(likesList)
    save_progress(countLikesCrawling)
        

now is on number 5 crawling

In [102]:
def read_friends_csv(countLikesCrawling): #read the friends from the file 
    i = 0
    with open("./data/friends.csv","r") as file:
        friendList = []
        for line in file:
            if i in range(0,1002):
                friendList.append(line)
            elif i>1002:
                break
            i += 1
    return friendList
friendList = read_friends_csv(countLikesCrawling)
len(friendList)
with open("./data/friends_already_done.csv","w") as file:
    for f in friendList:
        file.write(f+"\n")

In [73]:
with open("./data/friends.csv","r") as file:
    data = []
    for line in file:
        data.append(line)
len(data)


10000

In [98]:
with open("./data/likes.csv","r") as file:
    data = []
    for line in file:
        jsondata = json.loads(line)
        data.append(jsondata)
len(data)

1001

In [99]:
with open("./data/progress.csv","r") as file:
    print(file.readline())


1001
